# IMPACT DE LA SITUATION SANITAIRE SUR L'EMISSION ACOUSTIQUE DE LA ROCADE

In [73]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys, os
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import Bruit.Emission as be
import Import_trafics as it
import altair as alt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Méthode Générale
Importer les données de la DIRA, calculer le niveau d'émission horaire, visualiser  : 
- les ecarts entre les differents jour
- les ecarts entre les différentes situations

> ### Import des données DIRA
la DIRA nous a fourni des données sur les stations SE43 et SE24 
**ATTENTION** : les données de la station SE24 et sur les bretelles ne fournissent pas de vitesse ni de distinction VL / PL, donc on va partir sur une estimation. 
> - Dans un premier temps on traite d'abord les données de débit de la station S43, on les visualise et on analyse : on voit bien avec des valeurs fofaitaire de vitesse la différence du au trafic entre les jours ouvrés et le WE
> - il faut pas faire la somme des sens au début, mais on fera la somme energétiques des niveaux de bruit des deux sens pour chaque heure à la fin (comme ça si un sens est vraiment sup on ne le minore pas avec une vitesse moyenne)
> - ensuite on cherche a merger avec les données de vitesse, mais là pas de scripts tout prets dans les données de OTV. En plus les données de vitesses sont fournies uniquement par moyenne sur les jours ouvrés, **donc il faudra voir avec la DIRA comment récupérer des vitesses moyennes horaires par jour et par sens. est-ce possible ?**

>> #### Données de trafic

In [227]:
dossier=r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\PPBE BdxM\Bruit_ressenti\donnees_sources\DIRA\Comparaison_trafic_confinement-courvre-feu\Cerema_ech_24'
nomFichier='S43_debit_VL_PL_23_11_29_11.xls'
dfFichier=pd.read_excel(os.path.join(dossier, nomFichier),sheet_name=None)

In [249]:
#fonction utile pour pus tard jointure avec vitesses
def typeJour(date):
    if date.weekday() in range(5) : 
        return 'JO'
    elif date.weekday()==5 :
        return 'Samedi'
    else : 
        return 'Dimanche'
#utilisation des codes OTV
dira=it.Comptage_Dira(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRA\0_tmja_dira_par_section_20200106.ods',
                      r'C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\PPBE BdxM\Bruit_ressenti\donnees_sources\DIRA\Comparaison_trafic_confinement-courvre-feu\Cerema_ech_24',
                     '2019','local_otv_gti', 'na_2010_2019_p')
#Creer les données horaires format OTV, puis la passer pour un format de calcul
dfHoraireOtv=dira.miseEnFormeFichier(nomFichier,6)
dfHoraireOtv['typeJour']=dfHoraireOtv.jour.apply(lambda x : typeJour(x)) 
dfHoraireOtv['sens']=dfHoraireOtv.voie.apply(lambda x : x.lower().replace(' ',''))
dfHoraireOtv.set_index(['jour', 'type_veh','sens','typeJour'],inplace=True)
dfHoraireStack=dfHoraireOtv[[c for c in dfHoraireOtv.columns if c[0]=='h']].stack().reset_index().rename(columns={'level_4':'heure',0:'nbVeh'})
dfHoraireGroup=dfHoraireStack.groupby(['jour','heure','type_veh']).nbVeh.sum().reset_index()
dfHoraireEmission=dfHoraireGroup.loc[dfHoraireGroup.type_veh=='VL'].merge(dfHoraireGroup.loc[dfHoraireGroup.type_veh=='PL'][['jour','heure','nbVeh']], on=['jour','heure'],suffixes=('VL','PL'))
#mettre en forme les données
dfHoraireEmission['emission']=dfHoraireEmission.apply(lambda x : be.Route(x.nbVehVL, x.nbVehPL, 90, 80).lwm, axis=1)
dfHoraireEmission['dateHeure']=dfHoraireEmission.apply(lambda x : pd.to_datetime(f"{x.jour.strftime('%Y-%m-%d')} {x.heure.split('_')[0][1:]}:00:00"), axis=1)

In [216]:
#visualisation d'une semaine pour vérifier les niveaux  : on distingue bien le samedi et le dimanche, pour lesquels les niveaux sont 
alt.Chart(dfHoraireEmission).mark_line().encode(
    x='hours(dateHeure):T',
    y=alt.Y('emission:Q',scale=alt.Scale(zero=False)),
    color=alt.Color('day(dateHeure):T', scale=alt.Scale(scheme='tableau10'),sort=[0,1,2]),
    )

alt.Chart(...)

>> #### Données de vitesse

In [189]:
fichierVts='S43_vitesses_23_11_29_11.xls'
dfVts=pd.read_excel(os.path.join(dossier, fichierVts),sheet_name=None, skiprows=8, nrows=24)

In [190]:
listDfVts=[]
for i,s in ((0,'sensexter'),(1,'sensinter')) : 
    dFSensComplet=[dfVts[k] for k in dfVts.keys() if not 'Voie' in k and k[:2]!="xx"][i]
    listDfVts.append(pd.concat([dFSensComplet.iloc[:,[0,12,15]].rename(columns={'vitesse moyenne':'vMoy'}).assign(typeJour='JO', sens=s), 
           dFSensComplet.iloc[:,[0,28,31]].rename(columns={'vitesse moyenne.1':'vMoy','V85.1':'V85'}).assign(typeJour='Samedi', sens=s),
           dFSensComplet.iloc[:,[0,44,47]].rename(columns={'vitesse moyenne.2':'vMoy','V85.2':'V85'}).assign(typeJour='Dimanche', sens=s)]))
dFVtsFinale=pd.concat(listDfVts)
dFVtsFinale['sequence']=dFVtsFinale.sequence.apply(lambda x : 'h'+x.replace('-','_')[:-1])
dFVtsFinale.rename(columns={'sequence':'heure'}, inplace=True)

>> #### Jointure des trafic et vitesses

In [295]:
#joindre puis calcule de l'emission comme précédemment, mais pour chaque sens en moyennant au préalable le trafic par type de jour
dfHoraireStack.nbVeh.fillna(0, inplace=True)
dfGroupTypeJour=dfHoraireStack.groupby(['typeJour','heure','type_veh', 'sens']).nbVeh.mean().reset_index()
dFSensHeureJour=dfGroupTypeJour.loc[dfGroupTypeJour.type_veh=='VL'].merge(dfGroupTypeJour.loc[dfGroupTypeJour.type_veh=='PL'][['typeJour','heure','nbVeh','sens']], 
                                                        on=['typeJour','heure','sens'],suffixes=('VL','PL')).merge(dFVtsFinale, on=['typeJour','heure','sens']).drop('type_veh', axis=1)
dFSensHeureJour['emission']=dFSensHeureJour.apply(lambda x : be.Route(x.nbVehVL, x.nbVehPL, x.vMoy,x.vMoy-5).lwm, axis=1)
dfEmissionSensHeureJour=dFSensHeureJour.loc[dFSensHeureJour.sens=='sensexter'][['heure','typeJour','emission']].merge(dFSensHeureJour.loc[dFSensHeureJour.sens=='sensinter'][['heure','typeJour','emission']],
    on=['heure','typeJour'],suffixes=('Exter','Inter'))
dfEmissionSensHeureJour['emission_tot']=dfEmissionSensHeureJour.apply(lambda x : be.sommeEnergetique(x.emissionExter,x.emissionInter), axis=1) 

In [321]:
alt.Chart(dfEmissionSensHeureJour.assign(heure=dfEmissionSensHeureJour.heure.apply(lambda x : int(x.split('_')[0][1:])))).mark_line().encode(
    x=alt.X('heure', sort='ascending', scale=alt.Scale(domain=(0,23))),
    y=alt.Y('emission_tot:Q',scale=alt.Scale(zero=False)),
    color='typeJour')

alt.Chart(...)